# View light-Curves

- author Sylvie Dagoret-Campagne
- creation date 2024-09-03
- last update 2024-09-03

In [1]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

In [3]:
from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [4]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
path_photom ="data/photom/sources_objectTable"
tract = 3864

#inputfilename ="objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069-tract3864.csv"
inputfilename = "objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146-tract3864.csv"
inputfullfilename= os.path.join(path_photom,inputfilename) 

df_allsources = pd.read_csv(inputfullfilename,index_col=0)

In [5]:
df_allsources

,sourceId,psfFlux,psfFluxErr,psfFlux_flag,psfFlux_apCorr,psfFlux_apCorrErr,extendedness,detect_isPrimary,deblend_skipped,gaussianFlux,gaussianFluxErr,gaussianFlux_flag,localPhotoCalib,localPhotoCalibErr,localPhotoCalib_flag,index,visit,detector,ra,dec,physical_filter,band,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,x,y,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,apFlux_50_0_flag,localBackground_instFlux,localBackground_flag,source_row,obj_index,psfSn,psfMag,psfMagErr,apFlux_35_0_calFlux,apFlux_35_0_calFluxErr,apFlux_35_0_calMag,apFlux_35_0_calMagErr,apFlux_50_0_calFlux,apFlux_50_0_calFluxErr,apFlux_50_0_calMag,apFlux_50_0_calMagErr,psfInstMag,psfInstMagErr,psfGausFluxRatio,ccd_ccdVisitId,ccd_visitId,ccd_band,zenithDistance,zeroPoint,psfSigma,skyBg,skyNoise,astromOffsetMean,astromOffsetStd,nPsfStar,psfStarDeltaE1Median,psfStarDeltaE2Median,psfStarDeltaE1Scatter,psfStarDeltaE2Scatter,psfStarDeltaSizeMedian,psfStarDeltaSizeScatter,psfStarScaledDeltaSizeScatter,psfTraceRadiusDelta,maxDistToNearestPsf,effTime,effTimePsfSigmaScale,effTimeSkyBgScale,effTimeZeroPointScale,decl,ccd_detector,seeing,skyRotation,expMidpt,expMidptMJD,expTime,obsStart,obsStartMJD,darkTime,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec,airmass,medianE,dra,ddec,psfMagDiffMmag,psfMagDiffChi,raDiffMas,decDiffMas,raCosDecDiffMas,localPhotoCalibMag,localPhotoCalibMagErr,DZP
0,179300073413279748,1.580155e+06,12859.257891,False,1.024903,0.0,0.0,True,False,1.623147e+06,16479.172165,False,44.601713,0.088640,False,3,2023121500478,0,122.616310,-36.190488,SDSSz_65mm~empty,z,35950.336609,617.848502,3577.161510,50.744821,37540.276080,870.651651,False,0.283571,False,3,520,122.880748,15.903251,0.008836,1.603447e+06,27557.101564,15.887364,0.018660,1.674361e+06,38832.555057,15.840377,0.025181,20.026629,0.008836,0.973513,42748468736,2023121500478,z,21.000440,27.295220,4.229468,13.34357,8.061932,0.027840,0.027840,58,-0.002926,0.000457,0.017165,0.014529,-0.000798,0.063468,0.015093,0.640831,2102.7854,0.287298,0.055902,2.248274,0.076196,-36.139215,0,0.953239,359.999731,2023-12-16 05:39:06.498994,60294.235492,30.0,2023-12-16 05:38:51.498994,60294.235318,30.2444,4071,3999,122.498487,-36.193057,122.632616,-36.191674,122.630816,-36.085337,122.496869,-36.086718,1.071148,0.002961,0.051613,-0.051273,-289.071008,-6.930543,41.679035,-14.658130,16.119924,27.276621,0.002158,-0.018598
1,179300073413279762,1.722274e+06,13896.409761,False,1.008761,0.0,0.0,True,False,1.720439e+06,15854.387410,False,43.825502,0.088640,False,17,2023121500478,0,122.585459,-36.189035,SDSSz_65mm~empty,z,38142.817947,714.174432,2641.422054,117.690518,36449.190848,1007.305025,False,-0.155558,False,17,270,123.936603,15.809745,0.008760,1.671628e+06,31299.052698,15.842151,0.020329,1.597404e+06,44145.647972,15.891463,0.030005,19.914062,0.008760,1.001067,42748468736,2023121500478,z,21.000440,27.295220,4.229468,13.34357,8.061932,0.027840,0.027840,58,-0.002926,0.000457,0.017165,0.014529,-0.000798,0.063468,0.015093,0.640831,2102.7854,0.287298,0.055902,2.248274,0.076196,-36.139215,0,0.953239,359.999731,2023-12-16 05:39:06.498994,60294.235492,30.0,2023-12-16 05:38:51.498994,60294.235318,30.2444,4071,3999,122.498487,-36.193057,122.632616,-36.191674,122.630816,-36.085337,122.496869,-36.086718,1.071148,0.002961,0.020762,-0.049820,-190.569969,-6.295061,-2.146178,-3.282275,-4.634251,27.295683,0.002196,0.000463
2,179300073413279801,1.061165e+07,34913.067517,False,1.015366,0.0,0.0,True,False,1.067569e+07,39481.891718,False,44.669276,0.088640,False,56,2023121500478,0,122.614697,-36.183513,SDSSz_65mm~empty,z,236933.203217,805.869427,3531.742421,313.696491,241064.481355,1040.035058,False,1.082012,False,56,654,303.944893,13.835543,0.003572,1.058363e+07,35997.603672,13.838413,0.003693,1.076818e+07,46457.612786,13.819645,0.004684,17.960565,0.003572,0.994001,42748468736,2023121500478,z,21.000440,27.295220,4.229468,13.34357,8.061932,0.027840,0.027840,58,-0.002926,0.000457,0.017165,0.014529,-0.000798,0.063468,0.015093,0.64083

In [13]:
len(df_allsources.obj_index.unique())

367

In [15]:
list_of_files = os.listdir(path_photom) 

In [27]:
all_objnum = []
tag = "^sources_objectTable-t3864-multibands-o(.*)-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv$"
for idx,file in enumerate(list_of_files):
    coll = re.findall(tag,file)
    if len(coll)>0:
        all_objnum.append(int(coll[0]))
        
all_objnum = sorted(all_objnum)  

In [30]:
all_df = []
for idx,idx_obj in enumerate(all_objnum):
    filename = f"sources_objectTable-t3864-multibands-o{idx_obj}-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv"
    fullfilename = os.path.join(path_photom,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    all_df.append(df)
    

In [31]:
df = all_df[0]

In [32]:
df

,sourceId,psfFlux,psfFluxErr,psfFlux_flag,psfFlux_apCorr,psfFlux_apCorrErr,extendedness,detect_isPrimary,deblend_skipped,gaussianFlux,gaussianFluxErr,gaussianFlux_flag,localPhotoCalib,localPhotoCalibErr,localPhotoCalib_flag,index,visit,detector,ra,dec,physical_filter,band,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,x,y,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,apFlux_50_0_flag,localBackground_instFlux,localBackground_flag,source_row,obj_index,psfSn,psfMag,psfMagErr,apFlux_35_0_calFlux,apFlux_35_0_calFluxErr,apFlux_35_0_calMag,apFlux_35_0_calMagErr,apFlux_50_0_calFlux,apFlux_50_0_calFluxErr,apFlux_50_0_calMag,apFlux_50_0_calMagErr,psfInstMag,psfInstMagErr,psfGausFluxRatio,ccd_ccdVisitId,ccd_visitId,ccd_band,zenithDistance,zeroPoint,psfSigma,skyBg,skyNoise,astromOffsetMean,astromOffsetStd,nPsfStar,psfStarDeltaE1Median,psfStarDeltaE2Median,psfStarDeltaE1Scatter,psfStarDeltaE2Scatter,psfStarDeltaSizeMedian,psfStarDeltaSizeScatter,psfStarScaledDeltaSizeScatter,psfTraceRadiusDelta,maxDistToNearestPsf,effTime,effTimePsfSigmaScale,effTimeSkyBgScale,effTimeZeroPointScale,decl,ccd_detector,seeing,skyRotation,expMidpt,expMidptMJD,expTime,obsStart,obsStartMJD,darkTime,xSize,ySize,llcra,llcdec,ulcra,ulcdec,urcra,urcdec,lrcra,lrcdec,airmass,medianE,dra,ddec,psfMagDiffMmag,psfMagDiffChi,raDiffMas,decDiffMas,raCosDecDiffMas,localPhotoCalibMag,localPhotoCalibMagErr,DZP
500,178737290963583254,2.765022e+06,15732.080763,False,0.996067,0.0,0.0,True,False,2.752271e+06,18712.172330,False,40.851923,0.065515,False,277,2023112900634,0,122.458937,-36.203402,SDSSr_65mm~empty,r,69453.830451,617.503044,2380.238795,2116.264594,72059.884913,845.132886,False,0.354473,False,277,64,175.756939,15.295753,0.006177,2.837323e+06,25226.186702,15.267728,0.009653,2.943785e+06,34525.303450,15.227735,0.012734,19.323785,0.006177,1.004633,42614290944,2023112900634,r,6.855807,27.375334,5.127872,11.407619,7.994438,0.019821,0.019821,71,0.004502,-0.008782,0.025208,0.023649,0.027897,0.070267,0.013667,0.880071,1796.8524,0.264971,0.038030,2.629821,0.088313,-36.206625,0,1.155550,0.000186,2023-11-30 08:02:17.118006,60278.334920,30.0,2023-11-30 08:02:02.118006,60278.334747,30.2437,4071,3999,122.381401,-36.260475,122.515604,-36.259077,122.513777,-36.152738,122.379757,-36.154134,1.007202,0.009869,0.011303,0.003223,-32.607923,-2.650750,-12.165692,28.572796,25.966862,27.371969,0.001741,-0.003365
517,178737292037325173,3.796435e+06,23732.628633,False,1.008703,0.0,0.0,True,False,3.807841e+06,28966.492744,False,67.631085,0.124960,False,372,2023112900635,0,122.458931,-36.203406,SDSSz_65mm~empty,z,56130.541188,613.347312,2383.284906,2110.487447,57039.628103,844.669879,False,0.185647,False,372,64,159.966879,14.951560,0.006787,3.796169e+06,41481.344477,14.951636,0.011864,3.857652e+06,57125.940802,14.934192,0.016078,19.526926,0.006787,0.997004,42614291200,2023112900635,z,6.779472,26.827257,4.439398,12.141253,7.992703,0.020959,0.020959,83,-0.013753,-0.004893,0.046487,0.024159,0.009282,0.061625,0.013826,0.825472,1055.9867,0.121030,0.050740,2.470915,0.032178,-36.206477,0,1.000500,0.000059,2023-11-30 08:03:03.847495,60278.335461,30.0,2023-11-30 08:02:48.847495,60278.335288,30.2308,4071,3999,122.381286,-36.260335,122.515496,-36.258935,122.513673,-36.152582,122.379645,-36.153980,1.007041,0.014597,0.011406,0.003071,-376.801088,-29.860701,-34.037874,13.769148,-10.370638,26.824634,0.002006,-0.002623
571,179300079855730924,1.796155e+06,10096.104952,False,1.004340,0.0,0.0,True,False,1.787261e+06,11891.257391,False,22.695517,0.059906,False,235,2023121500484,0,122.458938,-36.203409,SDSSg_65mm~empty,g,80049.336172,600.286968,2174.069843,2550.197946,83626.365591,813.039061,False,0.430589,False,235,64,177.905722,15.764141,0.006103,1.816761e+06,13623.822913,15.751755,0.008142,1.897944e+06,18452.341576,15.704292,0.010556,19.153991,0.006103,1.004976,42748470272,2023121500484,g,19.693762,28.007408,5.820602,3.049098,7.545565,0.015693,0.015693,43,0.004369,0.000963,0.016659,0.012421,0.000848,0.080857,0.013523,0.8

In [34]:
len(df_allsources[df_allsources.obj_index == all_objnum[0]] )

958

In [39]:
for idx,df in enumerate(all_df):
    obj_num = df.obj_index.unique()[0]
    
    df_filt = {}
    for b in ["g","r","i","z","y"]:
        df_filt[b] = df[df.band == b]
    break
    

In [43]:
print(list(df_filt["g"].columns))

['sourceId', 'psfFlux', 'psfFluxErr', 'psfFlux_flag', 'psfFlux_apCorr', 'psfFlux_apCorrErr', 'extendedness', 'detect_isPrimary', 'deblend_skipped', 'gaussianFlux', 'gaussianFluxErr', 'gaussianFlux_flag', 'localPhotoCalib', 'localPhotoCalibErr', 'localPhotoCalib_flag', 'index', 'visit', 'detector', 'ra', 'dec', 'physical_filter', 'band', 'apFlux_35_0_instFlux', 'apFlux_35_0_instFluxErr', 'x', 'y', 'apFlux_50_0_instFlux', 'apFlux_50_0_instFluxErr', 'apFlux_50_0_flag', 'localBackground_instFlux', 'localBackground_flag', 'source_row', 'obj_index', 'psfSn', 'psfMag', 'psfMagErr', 'apFlux_35_0_calFlux', 'apFlux_35_0_calFluxErr', 'apFlux_35_0_calMag', 'apFlux_35_0_calMagErr', 'apFlux_50_0_calFlux', 'apFlux_50_0_calFluxErr', 'apFlux_50_0_calMag', 'apFlux_50_0_calMagErr', 'psfInstMag', 'psfInstMagErr', 'psfGausFluxRatio', 'ccd_ccdVisitId', 'ccd_visitId', 'ccd_band', 'zenithDistance', 'zeroPoint', 'psfSigma', 'skyBg', 'skyNoise', 'astromOffsetMean', 'astromOffsetStd', 'nPsfStar', 'psfStarDeltaE1

In [44]:
df_filt["g"]['obsStart']

571      2023-12-16 05:45:03.673001
625      2023-12-16 05:48:58.151995
660      2023-12-16 05:52:51.169990
874      2024-01-31 03:56:46.173999
1097     2023-12-16 06:15:59.652996
                    ...            
79667    2024-03-22 03:43:54.087002
80803    2024-04-04 02:31:04.206996
82286    2023-11-30 06:44:08.611503
82791    2023-11-30 07:11:50.277008
83164    2024-04-04 04:16:24.033494
Name: obsStart, Length: 296, dtype: object